# Numerical simulation of magnetic particles on a plane

In this notebook we try to mimick the analog particle simulator using Metropolis-Hastings Monte Carlo (MC) simulation.
We will use the software [_Faunus_](https://faunus.readthedocs.io) which is designed to handle a broad range of
molecular systems, spanning from atomistic models to coarse grained particle representations.

## Learning outcomes

- Be able to setup and run numerical particle simulations using pair-potentials.
- Understand the concept of periodic boundary conditions (PBC)
- Visualize simulation output
- Extract structural information from simulations, specifically the _radial distribution function_, $g(r)$.
- Use $g(r)$ to calculate thermodynamic properties

## Literature
- [_Intermolecular and Surface Forces_](https://www.sciencedirect.com/book/9780123751829/intermolecular-and-surface-forces) by J. Israelachvili
- [_Understanding Molecular Simulation_](http://www.sciencedirect.com/science/book/9780122673511)) by Frenkel and Smit

## Model System

The model system is a pseudo _cubic box_ with side lengths $L$ containing $N$ particles located on the xy-plane and translate in the _x_ and _y_ directions.
This effectively mimicks a planar surface as in the analog simulator.
Particles interact via $\propto$ $r^{-12}$ and $\propto$ $r^{-3}$ pair-potentials. 
The former models exchange repulsion between finite-sized particles:

$$
u_{rep}(r) = \epsilon \left (\frac{\sigma}{r} \right )^{12}
$$

where $\sigma$ is the particle's diameter, and $\epsilon$ is a positive constant.
The $r^{-3}$ potential described the magnetic dipole-dipole repulsion (↑↑):

$$
u_{d-d}(r) = \frac{C}{r^{3}}
$$

where $C$ is a constant proportional to the squared magnitude of the particle's magnetic dipole moment, see e.g. the book of Israelachvili, page 83.
In molecular simulations, the boundary effects related to the small size of the simulation box compared to bulk systems is often handled using periodic boundary conditions (PBC): any particle passing through one boundary of the simulation cell comes back into the cell through the opposite boundary (page 34, Frenkel & Smit).
If we do not use PBC in our simulation, our system would be confined by hard walls corresponding to the repulsive magnetic bars at the sides of the "analog" 2D particle simulator.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import json
import mdtraj as md
import base64
from scipy.spatial import distance
from scipy.interpolate import Akima1DInterpolator
from IPython.display import HTML
from cryptography.fernet import Fernet

plt.rcParams.update({"font.size": 16, "figure.figsize": [8.0, 6.0]})


def encrypt(string, key):
    """Helper function to encrypt a string with a password"""
    def keygen(x):
        return base64.urlsafe_b64encode(x.encode() + b" " * (32 - len(x)))
    cipher = Fernet(keygen(key))
    return cipher.encrypt(string.encode())


def decrypt(string, key):
    """Helper function to decrypt a string"""
    def keygen(x):
        return base64.urlsafe_b64encode(x.encode() + b" " * (32 - len(x)))
    cipher = Fernet(keygen(key))
    return print(cipher.decrypt(string.encode()).decode())

## Monte Carlo simulations

The following cells generate input to simulate the system with and without periodic boundary counditions.

- `mcloop` determines the number of MC sweeps (_macro_ ✕ _micro_).
- `atomlist` indicates the type of atoms in the system. We are going to simulate magnetic particles (*MP*). The [trial moves](https://faunus.readthedocs.io/en/latest/_docs/moves.html) for these particles are translations in the xy-plane and the length of the displacement is determined by the product of *dp* times a random unit vector.
- in `insertmolecules` we set the number of magnets in the system.
- in `energy` we set the nonbonded interactions. _sigma_ sets the particle diameter, _prefactor_ sets the constant for the $r^{-3}$-interaction (*C*) while lj-prefactor sets the energy parameter for the $r^{-12}$-interaction ($\epsilon$). In the case of simulations with hard walls, we add an additional term to the [Hamiltonian](https://faunus.readthedocs.io/en/latest/_docs/energy.html) which confines the particles in $-L/2\leq x\leq L/2$ and $-L/2\leq y\leq L/2$. This term is a quadratic function of the distance from the boundary with spring constant $k$ that is applied to the particle when it is outside the boundaries.
- in [`analysis`](https://faunus.readthedocs.io/en/latest/_docs/analysis.html) we indicate the analysis that we want to perform during the simulation. Here, we specify that we want to save each configuration into a trajectory file and the final configuration into a `gro` file. We also want to print to file the energy of the system (total, nonbonded, or confinement) for each frame.

Trajectory files are saved in `xtc` format – notice that the default length unit in _Faunus_ is Ångstrom, while coordinates are saved in nm in `xtc` and `gro` files.

### Input Parameters

In [ ]:
sigma = 53.0  # particle diameter (Å)
dp = 100.0  # particle displacement (Å)
N = 55  # number of particles
L = 600.0  # Side length of the box (Å)
C = 1e6  # Magnetic dipole interaction prefactor, C (1e6)
epsilon = 10  # Lennard-Jones prefactor (10)
micro = 1e4  # number of MC sweeps
trajname = f"traj_{N}" # trajectory file name
confname = f"conf_{N}" # final configuration file name
energyname = f"energy_{N}" # energy file name

### Simulation _with_ periodic boundaries conditions (PBC)

In [ ]:
js = {
    "temperature": 298,
    "mcloop": {"macro": 1, "micro": micro},
    "geometry": {"type": "cuboid", "length": L},
    "energy": [
        {
            "nonbonded": {
                "default": [
                    {
                        "repulsionr3": {
                            "prefactor": C,
                            "lj-prefactor": epsilon,
                            "sigma": sigma,
                        }
                    }
                ]
            }
        }
    ],
    "atomlist": [{"MP": {"dp": dp}}],
    "moleculelist": [
        {"magnets": {"atoms": ["MP"], "atomic": True, "insdir": [1, 1, 0]}}
    ],
    "insertmolecules": [{"magnets": {"N": N}}],
    "moves": [{"transrot": {"molecule": "magnets", "dir": [1, 1, 0]}}],
    "analysis": [
        {"xtcfile": {"file": trajname + "_pbc.xtc", "nstep": 10}},
        {"savestate": {"file": confname + "_pbc.gro"}},
        {"systemenergy": {"file": energyname + "_pbc.dat", "nstep": 10}},
    ],
}
with open("pbc.json", "w+") as f:
    print('Writing input file "pbc.json" to disk...')
    f.write(json.dumps(js, indent=4))
    
!faunus -i pbc.json

### Simulation _without_ periodic boundaries (hard walls)

We now setup a simulation of particles on a square with hard walls, i.e. no periodic boundaries.
This is closer to the "analog" particle simulation from the previous Notebook.

In [ ]:
js = {
    "temperature": 298,
    "mcloop": {"macro": 1, "micro": micro},
    "geometry": {"type": "cuboid", "length": L},
    "energy": [
        {
            "nonbonded": {
                "default": [
                    {
                        "repulsionr3": {
                            "prefactor": C,
                            "lj-prefactor": epsilon,
                            "sigma": sigma,
                        }
                    }
                ]
            }
        },
        {
            "confine": {
                "type": "cuboid",
                "low": [-L / 2 + sigma, -L / 2 + sigma, 0],
                "high": [L / 2 - sigma, L / 2 - sigma, 0],
                "molecules": ["magnets"],
                "k": 1e6,
            }
        },
    ],
    "atomlist": [{"MP": {"dp": dp}}],
    "moleculelist": [
        {"magnets": {"atoms": ["MP"], "atomic": True, "insdir": [1, 1, 0]}}
    ],
    "insertmolecules": [{"magnets": {"N": N}}],
    "moves": [{"transrot": {"molecule": "magnets", "dir": [1, 1, 0]}}],
    "analysis": [
        {"xtcfile": {"file": trajname + "_nopbc.xtc", "nstep": 10}},
        {"savestate": {"file": confname + "_nopbc.gro"}},
        {"systemenergy": {"file": energyname + "_nopbc.dat", "nstep": 10}},
    ],
}
with open("nopbc.json", "w+") as f:
    f.write(json.dumps(js, indent=4))

!faunus -i nopbc.json

### Tasks and questions:
1. Why do you initially get a _humongous negative energy change_?
2. What does the _relative energy drift_ mean?
3. What is the area per particle? Give your answer in $Å^2$.

### Loading the Trajectory

We now load the Gromacs-style trajectory `xtc`-file using the library [MDTraj](https://www.mdtraj.org).
The coordinates of all the particles for all the frames can be accessed as an array of shape: *number of frames*, *number of particle*, *number of cartesian coordinates*.
The `xtc` format stores all lengths in nanometers.

In [ ]:
traj = md.load_xtc(trajname + "_pbc.xtc", top=confname + "_pbc.gro")
print(f"Number of particles = {traj.n_atoms}")
print(f"Number of frames    = {traj.n_frames}")
print(f"xyz shape           = {traj.xyz.shape}")

Here are the positions of the first three magnets in the 12th frame:

In [ ]:
traj.xyz[12, :3, :]

Here are the x-coordinates of the first 5th–7th magnets in the 20th frame:

In [ ]:
traj.xyz[20, 4:7, 0]

We can visualize the positions of the particles in the 20th frame by a simple xy plot:

In [ ]:
fig, ax = plt.subplots()
ax.plot(traj.xyz[10, :, 0] * 10, traj.xyz[10, :, 1] * 10, "ro", ms=15)
ax.set_aspect("equal")
ax.set_xlabel(r"distance")
ax.set_ylabel(r"distance")
ax.set_ylim(-10, 610)
ax.set_xlim(-10, 610)
plt.show()

We can also visualize trajectories using animated plots

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_ylim(-10, 610)
ax.set_xlim(-10, 610)
(line,) = ax.plot([], [], color="r", marker="o", lw=0, ms=15)


def init():
    line.set_data([], [])
    return (line,)


def animate(i):
    x = traj.xyz[i, :, 0] * 10
    y = traj.xyz[i, :, 1] * 10
    line.set_data(x, y)
    return (line,)


def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim.to_jshtml())


anim = animation.FuncAnimation(
    fig,
    animate,
    init_func=init,
    frames=range(0, traj.n_frames, 20),
    interval=200,
    blit=True,
)
display_animation(anim)

### Calculate histogram for uniform distribution (No PBC = Hard Walls)

We now generate the distribution of particle distances for the case where these are completely uncorrelated, i.e. for an ideal 2D gas confined on a square with hard boundaries. This is done by randomly placing a large number of particles (the larger the number, the higher the accuracy) and build a histogram of distances.

In [ ]:
def ideal_histogram(N):
    """returns the uncorrelated (ideal) histogram of particle separations on a square"""
    x = np.random.randint(0, L + 1, N)
    y = np.random.randint(0, L + 1, N)
    pos = np.array([x, y]).T
    hist = np.histogram(distance.pdist(pos), bins=150, density=True)
    r = hist[1][1:]  # / diameter
    P = hist[0]
    return r, P


def real_histogram(traj):
    """returns the real histogram as sampled from a MC trajectory"""
    pairs = traj.top.select_pairs("name MP", "name MP")
    d = md.compute_distances(traj, atom_pairs=pairs, periodic=False, opt=True) * 10
    hist = np.histogram(d, bins=150, density=True)
    r = hist[1][1:]  # / diameter
    P = hist[0] * len(pairs) / (0.5 * N**2)
    return r, P

### Calculate interparticle distances from the MC trajectory of the system without PBC

Here we are loading a trajectory for a system of *N* particles confined on a square by hard walls.
Interparticle distances for all frames are calculated using the trajectory analysis `MDTraj` function [`compute_distances`](http://mdtraj.org/1.9.0/api/generated/mdtraj.compute_distances.html#mdtraj.compute_distances).

In [ ]:
traj = md.load_xtc(trajname + "_nopbc.xtc", top=confname + "_nopbc.gro")
r, P = real_histogram(traj)
plt.plot(r, P, lw=2, label="Monte Carlo")

r, P = ideal_histogram(5000)
plt.plot(r, P, lw=2, label="Ideal")

plt.xlabel(r"Particle-particle distance, $r$")
plt.ylabel(r"Probability, $P(r)$")
plt.legend(frameon=False, loc=0)
plt.show()

### Calculate histogram for a PBC with minimum image convention

We now generate the distribution of particle distances for the case where these are uncorrelated, i.e. for an ideal 2D gas with PBC using the minimum image convention. This is done by (i) generating pairs of random numbers (x- and y-coordinates), (ii) calculating the distances between the points, (iii) building a histogram of distances.
In the following cell we present a slow implementation, analogous to the one previously used for the system without PBC.

We can also get the _real_ distribution by loading a MC trajectory for a system of _N_ particles simulated with PBC. Interparticle distances for all frames are calculated using the trajectory analysis `MDTraj` function [`compute_distances`](http://mdtraj.org/1.9.0/api/generated/mdtraj.compute_distances.html#mdtraj.compute_distances).

In [ ]:
def ideal_histogram_pbc_slow(N):
    """Uncorrelated histogram of particle-particle distance using PBC (slow version)"""
    x = np.random.randint(0, L + 1, N)
    y = np.random.randint(0, L + 1, N)
    pos = np.array([x, y]).T

    def minimum_image(u, v):  # calculate distance according to minimum image convention
        dvec = np.absolute(u - v)
        d2 = 0
        for i in dvec:
            if i > L / 2:
                d2 += (i - L) ** 2
            else:
                d2 += i**2
        return np.sqrt(d2)

    P, r = np.histogram(
        distance.pdist(pos, lambda u, v: minimum_image(u, v)), bins=150, density=True
    )
    r = r[1:] - (r[1] - r[0]) / 2  # / diameter
    return r, P


def ideal_histogram_pbc(N):
    """Uncorrelated histogram of particle-particle distance using PBC"""
    x = np.random.randint(0, L + 1, N)
    y = np.random.randint(0, L + 1, N)
    pos = np.array([x, y]).T

    def distPBC(d1, d2, half_len):
        """Calculate minimum distance, r, according to the minimum image convention"""
        half_len = int(half_len)
        # print(d1.shape, d2[:,np.newaxis].shape)
        delta = np.abs(d1 - d2[:, np.newaxis])
        # print(delta.shape)
        delta[delta > half_len] -= half_len * 2
        return np.linalg.norm(delta, axis=2)

    P, r = np.histogram(
        distPBC(pos, pos, L / 2.0)[np.triu_indices(x.size, k=1)], bins=150, density=True
    )
    r = r[1:] - (r[1] - r[0]) / 2  # task: normalize by particle diameter
    return r, P


def real_histogram_pbc(traj):
    """real histogram of particle-particle distances using PBC on loaded MC trajectory"""
    pairs = traj.top.select_pairs("name MP", "name MP")
    d = md.compute_distances(traj, atom_pairs=pairs, periodic=True, opt=True) * 10
    hist = np.histogram(d, bins=150, density=True)
    r = hist[1][1:]  # divide by diameter
    P = hist[0] * len(pairs) / (0.5 * N**2)
    return r, P

In [ ]:
traj = md.load_xtc(trajname + "_pbc.xtc", top=confname + "_pbc.gro")
r, P = real_histogram_pbc(traj)
plt.plot(r, P, "r-", lw=2, label="Monte Carlo w. PBC")

r_ideal, P_ideal = ideal_histogram_pbc(3000)
plt.plot(r_ideal, P_ideal, "b-", lw=2, label="Ideal w. PBC")

plt.ylim(-0.0001, 0.0061)
plt.legend(frameon=False, loc="upper left")
plt.xlabel(r"Distance, $r$")  # add correct unit
plt.ylabel(r"Probability, $P(r)$")
plt.show()

### Calculate final distribution function, $g(r) = P_{real}(r) / P_{ideal}(r)$

In order to _divide_ the simulated distribution with the one for uncorrelated particles, the data points in each set need to be aligned. This is achieved by _splining_ the probability distributions against the same distance vector using the interpolation method by [Akima](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.Akima1DInterpolator.html#scipy.interpolate.Akima1DInterpolator).

In [ ]:
rvals = np.linspace(0, L, 300)

rdf = Akima1DInterpolator(r, P)(rvals) / Akima1DInterpolator(r_ideal, P_ideal)(rvals)
np.nan_to_num(rdf, copy=False)  # replace NaN with zero
plt.plot(rvals, rdf, "r-", lw=2)

plt.plot([0, 700], [1, 1], "k--", lw=0.5)
plt.ylim(-0.5, 3)
plt.xlabel(r"$r$")
plt.ylabel(r"$g(r)$")
plt.show()

### Calculate average energy 

Integrate the pair potential over the volume around a test particle weighting by the radial distribution function:

In [ ]:
area = L * L
u_rep = epsilon * (sigma / rvals) ** 12
u_dd = C / rvals**3
U_av = 0  # task: finish this
print(f"Average energy = {U_av}")

# password hint: something Japanese times two...
# decrypt('gAAAAABj9Ky9VPVC3xD1wJlD0Hqf4lpLjSkcm18x0pX08K0M8aSz_6cH-0B6EXaRIWgng3ldLrSBCzXueglL8R-fAGQxubv3BNvq7MmuQ69fl4xk7d6TxfDAF9C82MZdRl9prq8BKvm-huus448umYmSkWcs_m2k4S9Sl_uf7edaACkRHM33wy3mNtW1-6MkXogiWXAiINnr', 'insert_password')

In [ ]:
!head "$energyname"_nopbc.dat

In [ ]:
energy_nonbonded = np.loadtxt(energyname + "_nopbc.dat", usecols=(1))
energy_nonbonded.mean()

### Exercises
1. Why do you initially get very large energies in the MC simulations? Explore the `--state state.json` option to [reload from a previously saved state](https://faunus.readthedocs.io/en/latest/_docs/running.html?highlight=state#restarting).
0. Explore the particle displacement parameter ($dp$). How does this influence the mean square displacement ($\sqrt{r^2}$) and move acceptance (see `out.json`)? Does it influence the final result?
0. How does temperature enter a Monte Carlo simulation?
0. Run simulation with and without PBC for systems of various number of particles.
1. For all figures, label axes specifying the units.
1. For a RDF figure, try to normalize with the particle diameter, $\sigma$ to get _reduced units_ (See e.g. the Frenkel and Smit book)
0. Observe the  distance distribution, $P(r)$, for magnetic particles in a 2D system of side length 600 pixels, with PBC. Why does $P(r)$ increase linearly with $r$ for $r<300$ pixels? What $r$-dependence would you expect for a 3D system? (Hint: Read paragraph 17-1 in Hill's book - especially page 301. Please mention the difference between the infinitesimal area and volume elements in 2D and 3D, respectively.)
0. Observe the radial distribution functions, RDF or $g(r)$, obtained from the 2D particle simulator and from MC simulations of magnetic particles, with and without PBC. Comment on the differences between the three RDFs, and explain the deviation from unity at large separations observed in the simulation result (with and without PBC) and for the 2D particle simulator.  (Hint: In the 2D particle simulator the walls are magnetic bars, not just hard walls.)
0. Simulate $g(r)$ with PBC for non-interacting particles (N.B. both dipole-dipole and size-exclusion interactions should be vanishing). Although in theory it should, the curve may not be unity for all distances. Why?
0. List the differences between the particle simulator and the MC system. (Hint: discuss at least boundaries, interactions).
0. Extra: Complete the script in the last cell of the notebook to calculate the average energy of the system with the following integral:
\begin{equation}
\bar{U} = \frac{N^2}{2A} \int_0^\infty  2\pi r u(r) g(r) \mathrm{d}r
\end{equation}
0. Extra: Compare the energy calculated from the $g(r)$ with the average of the energies calculated every 10 frames during the simulation and saved in the `energy_N_pbc.dat` or `energy_N_nopbc.dat` files.
0. Extra: Can we study kinetics and diffusion properties from MC simulation results? If yes, briefly describe the procedure; if not, explain the reason.